In [ ]:
import praw
import random
import time
import pandas as pd
from datetime import datetime
import os

# Authenticate
reddit = praw.Reddit(
    client_id="M2QliqnzecrUUDJOisbwAA",
    client_secret="MbOsWe6py2DDJoPGgoGKqQEJWa2liw",
    user_agent="script:fetcher:v1.0 (by u/YOUR_USERNAME)"
)

# Choose a subreddit
subreddit = reddit.subreddit("MLQuestions")

# Initialize counters
ama_count = 0
post_count = 0

# Create data directory if it doesn't exist
data_dir = 'data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Create Excel filenames
timestamp_str = datetime.now().strftime('%Y%m%d_%H%M%S')
excel_filename = os.path.join(data_dir, f"reddit_metrics_{subreddit.display_name}_{timestamp_str}.xlsx")
comments_filename = os.path.join(data_dir, f"reddit_comments_{subreddit.display_name}_{timestamp_str}.xlsx")

# Create Excel writer for posts
with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
    columns = ['id', 'title', 'content', 'author', 'subreddit', 'subreddit_id', 'url', 
              'permalink', 'created_utc', 'num_comments', 'score', 'upvotes', 'downvotes',
              'upvote_ratio', 'is_self', 'link_flair_text', 'has_link', 'removed_by_mods']
    pd.DataFrame(columns=columns).to_excel(writer, index=False)

# Create Excel writer for comments
with pd.ExcelWriter(comments_filename, engine='openpyxl') as writer:
    comment_columns = [
        'post_id', 'comment_id', 'comment_author', 'comment_body', 'comment_score', 'comment_created_utc', 'comment_permalink'
    ]
    pd.DataFrame(columns=comment_columns).to_excel(writer, index=False)

# Helper function to get the next available row in an Excel sheet
def get_next_row(filename, sheet_name=0):
    if not os.path.exists(filename):
        return 1  # Only header exists
    try:
        df = pd.read_excel(filename, sheet_name=sheet_name)
        return len(df) + 1  # +1 for header
    except Exception:
        return 1

# Fetch top 10 hot posts
for post in subreddit.top(limit=10):
    # Generate random timeout between 10 and 30 seconds
    timeout = random.randint(10, 30)
    time.sleep(timeout)
    
    # Convert UTC timestamp to datetime
    created_utc = datetime.fromtimestamp(post.created_utc)
    
    # Print post details
    print("-" * 40)
    print(f"ID: {post.id}")
    print(f"Title: {post.title}")
    print(f"Content: {post.selftext}")
    print(f"Author: {post.author}")
    print(f"Subreddit: {post.subreddit}")
    print(f"Subreddit ID: {post.subreddit_id}")
    print(f"URL: {post.url}")
    print(f"Permalink: {post.permalink}")
    print(f"Created UTC (raw): {post.created_utc}")
    print(f"Created UTC (formatted): {created_utc}")
    print(f"Number of Comments: {post.num_comments}")
    print(f"Score: {post.score}")
    print(f"Upvotes: {post.ups}")
    print(f"Downvotes: {post.downs}")
    print(f"Upvote Ratio: {post.upvote_ratio}")
    print(f"Is Self Post: {post.is_self}")
    print(f"Link Flair Text: {post.link_flair_text}")
    print("-" * 40)
    
    # Create single post DataFrame
    post_df = pd.DataFrame([{
        'id': post.id,
        'title': post.title,
        'content': post.selftext,
        'author': str(post.author),
        'subreddit': str(post.subreddit),
        'subreddit_id': post.subreddit_id,
        'url': post.url,
        'permalink': post.permalink,
        'created_utc': created_utc,  # Store the datetime object instead of raw timestamp
        'num_comments': post.num_comments,
        'score': post.score,
        'upvotes': post.ups,
        'downvotes': post.downs,
        'upvote_ratio': post.upvote_ratio,
        'is_self': post.is_self,
        'link_flair_text': post.link_flair_text,
        'has_link': bool(post.url) and not post.is_self,
        'removed_by_mods': post.removed_by_category == "moderator"
    }])
    
    # Append to Excel file at the correct row
    post_startrow = get_next_row(excel_filename)
    with pd.ExcelWriter(excel_filename, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
        post_df.to_excel(writer, index=False, header=False, startrow=post_startrow)
    
    post_count += 1
    
    # Count AMA posts
    if post.link_flair_text and 'AMA' in post.link_flair_text.upper():
        ama_count += 1

    # --- Fetch top 5 comments for this post ---
    post.comments.replace_more(limit=0)
    top_comments = []
    for i, comment in enumerate(post.comments[:5]):
        if isinstance(comment, praw.models.Comment):
            comment_created_utc = datetime.fromtimestamp(comment.created_utc)
            comment_permalink = f"https://www.reddit.com{comment.permalink}"
            top_comments.append({
                'post_id': post.id,
                'comment_id': comment.id,
                'comment_author': str(comment.author),
                'comment_body': comment.body,
                'comment_score': comment.score,
                'comment_created_utc': comment_created_utc,
                'comment_permalink': comment_permalink
            })
    if top_comments:
        comments_df = pd.DataFrame(top_comments)
        comments_startrow = get_next_row(comments_filename)
        with pd.ExcelWriter(comments_filename, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            comments_df.to_excel(writer, index=False, header=False, startrow=comments_startrow)

print(f"\n✅ Data saved to {excel_filename}")
print(f"✅ Top comments saved to {comments_filename}")

# Read the final Excel file to get summary statistics
df = pd.read_excel(excel_filename)

# Print summary statistics
print("\n📈 Summary Statistics:")
print(f"Total Posts Analyzed: {post_count}")
print(f"Average Upvotes: {df['upvotes'].mean():.2f}")
print(f"Average Comments: {df['num_comments'].mean():.2f}")
print(f"Posts with Links: {df['has_link'].sum()}")
print(f"Posts Removed by Mods: {df['removed_by_mods'].sum()}")
print(f"AMA Posts: {ama_count}")

# Print unique timestamps to verify
print("\n📅 Timestamp Analysis:")
print(f"Number of unique timestamps: {df['created_utc'].nunique()}")
print("\nFirst few timestamps:")
print(df['created_utc'].head())
